# Lab 2

In [18]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote
from sqlalchemy import text
import pandas as pd

### Establish connection with database 'books'

In [19]:
server_name   = "localhost"
database_name = "books"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')


try:    
    engine = create_engine(url_string) # create engine object
    with engine.connect() as connection: # NOTE with-statement (Python manager form) connection object (is not used here)
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print(f'Error while connecting to database {database_name}:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=books;Trusted_Connection=yes 

Successfully connected to books!


### Input from user returning rows with relevant book titles, stock status and shop name

In [52]:
user_search = input('Enter book title')

query = ("SELECT b.BookTitle, i.BooksInStock, s.ShopName FROM Books b JOIN Inventory i ON b.BookId = i.BookId JOIN Shops s ON i.ShopId = s.ShopId WHERE b.BookTitle LIKE ('%' + :searched_title + '%')")

with engine.connect() as conn:
    print(f"Search string: {user_search} \n")
    print(f"Relevant books:")
    result = conn.execute(text(query), {"searched_title": user_search})
    for row in result:
        print(f"Title: {row.BookTitle}; Books in stock: {row.BooksInStock}, Shop: {row.ShopName}")

Search string: white 

Relevant books:
Title: The white elephant; Books in stock: 12, Shop: The Book Heaven
Title: Red, white, blue; Books in stock: 6, Shop: The Dusty Bookshelf
Title: The white elephant; Books in stock: 6, Shop: The Dusty Bookshelf
Title: Red, white, blue; Books in stock: 8, Shop: Twice Sold Tales
Title: The white elephant; Books in stock: 6, Shop: Twice Sold Tales


### Input from user returning dataframe with relevant book titles, stock status and shop name

In [53]:
user_search = input('Enter book title')

query = ("SELECT b.BookTitle, i.BooksInStock, s.ShopName FROM Books b JOIN Inventory i ON b.BookId = i.BookId JOIN Shops s ON i.ShopId = s.ShopId WHERE b.BookTitle LIKE ('%' + :searched_title + '%')")

print(f"Search string: {user_search} \n")
df = pd.read_sql_query(text(query), params={"searched_title": user_search}, con=engine)
df

Search string: white 



,BookTitle,BooksInStock,ShopName
0,The white elephant,12,The Book Heaven
1,"Red, white, blue",6,The Dusty Bookshelf
2,The white elephant,6,The Dusty Bookshelf
3,"Red, white, blue",8,Twice Sold Tales
4,The white elephant,6,Twice Sold Tales


---
---
### Basic query returning all rows

In [22]:
query = ("SELECT b.BookTitle, i.BooksInStock, s.ShopName FROM Books b JOIN Inventory i ON b.BookId = i.BookId JOIN Shops s ON i.ShopId = s.ShopId")

df = pd.read_sql_query(text(query), con=engine)
df

,BookTitle,BooksInStock,ShopName
0,What things?,32,The Book Heaven
1,Allt om yrsel,4,The Book Heaven
2,Too many reasons why,17,The Book Heaven
3,The white elephant,12,The Book Heaven
4,Things we hide,257,The Book Heaven
5,Things we find,7,The Book Heaven
6,100 reasons why,3,The Book Heaven
7,"Red, white, blue",6,The Dusty Bookshelf
8,What things?,8,The Dusty Bookshelf
9,Allt om yrsel,6,The Dusty Bookshelf
